In [1]:
import numpy as np
import functools as ft
import networkx as nx

In [2]:
def hdv(d):
    return np.random.choice([-1, 1], d)


def zero(d):
    return np.zeros((d,))


def bind(xs):
    return ft.reduce(lambda x, y: x * y, xs)


def bundle(xs):
    return ft.reduce(lambda x, y: x + y, xs)


def p(d):
    return np.random.shuffle(np.eye(d))


def inverse(P):
    return np.linalg.inv(P)


def permute(P, H):
    return P.dot(H)


def cosine_similarity(A, B):
    norm_A = np.linalg.norm(A)
    norm_B = np.linalg.norm(B)

    if norm_A == 0 or norm_B == 0:
        return 0

    return np.dot(A, B) / (norm_A * norm_B)


def similarity(A, B):
    return np.dot(A, B) / len(A)


sim = cosine_similarity

In [3]:
class ItemMemory:
    def __init__(self, vectors=[]):
        self.vectors = vectors

    def addVector(self, label, V):
        self.vectors.append((label, V))

    def getVector(self, label):
        return self.vectors[label]

    def cleanup(self, V):
        return max(self.vectors, key=lambda x: sim(V, x[1]))

In [4]:
def initVertices(graph, d=10000):
    for n in graph:
        graph.nodes[n]["hdv"] = hdv(d)

In [5]:
def initNodeMem(graph, memory):
    for n in graph:
        xs = map(lambda x: graph.nodes[x]["hdv"], list(graph.neighbors(n)))
        graph.nodes[n]["mem"] = bundle(xs)
        memory.addVector(f"mem{n}", graph.nodes[n]["mem"])

In [6]:
def retrain(graph, memory, threshold, iter=15):
    count = 0
    for i in range(iter):
        for n in graph:
            mem = graph.nodes[n]["mem"]
            finish = True
            for neighbor in map(
                lambda x: graph.nodes[x]["hdv"], list(nx.neighbors(graph, n))
            ):
                if sim(mem, neighbor) < threshold:
                    mem = bundle([mem, neighbor])
                    finish = False
                    print("here")

            for non_neighbor in map(
                lambda x: graph.nodes[x]["hdv"], list(nx.non_neighbors(graph, n))
            ):
                if sim(mem, non_neighbor) > threshold:
                    mem = bundle([mem, -non_neighbor])
                    finish = False
                    print("here")

            if finish:
                return

            graph.nodes[n]["mem"] = mem
            memory.addVector(f"mem{n}_{i}", mem)

In [7]:
def initGraph(graph, memory):
    G = []
    for n in graph.nodes():
        G.append(bind([graph.nodes[n]["hdv"], graph.nodes[n]["mem"]]))
    G = bundle(G) / 2
    memory.addVector("g", G)
    return G

In [8]:
def checkEdge(G, A, B, threshold):
    return sim(B, bind([G, A])) > threshold

In [9]:
def nodeMemoryReconstruction(G, xs, iter=15):
    if iter == 0:
        return list(map(lambda H: bind([G, H]), xs))

    mems = nodeMemoryReconstruction(G, xs, iter - 1)

    newMems = []
    b = [bind(x) for x in zip(mems, xs)]
    for i in range(len(xs)):
        bmem = bundle(b[:i] + b[i + 1 :])
        newMems.append(bind([xs[i], bundle([G, -bmem])]))

    return newMems

In [19]:
def graphMemoryReconstruction(G, xs, iter=15):
    if iter == 0:
        return (G, zeros(len(G)))

    (Gi, Ni) = graphMemoryReconstruction(G, xs, iter - 1)
    Gii = bundle([Gi, -Ni])

    for i in range(iter):
        print(i)

    return bundle([Gi, -edges])

---


# Tests


In [10]:
def testCheckEdge(G, graph, threshold):
    count = 0

    for n in graph.nodes():
        for m in graph.nodes():
            if n == m:
                continue
            exist = graph.has_edge(n, m)
            check = checkEdge(
                G, graph.nodes[n]["hdv"], graph.nodes[m]["hdv"], threshold
            )
            if exist != check:
                count += 1
                print(n, m, exist, check)

    print(count, "%.5f" % round(count / EDGES, 5))

In [11]:
def testNodeMemoryReconstruction(G, graph, iter=15):
    for i in range(iter):
        memsi = nodeMemoryReconstruction(
            G, list(map(lambda x: graph.nodes[x]["hdv"], graph.nodes())), iter=i
        )
        print(
            f"{0}_{i:02} =>",
            "%.10f" % abs(sim(graph.nodes[0]["mem"], memsi[0])),
        )

---


# Main


In [12]:
NODES, EDGES = 30, 150
DIMENSIONS, THRESHOLD, ITER = 3500, 0.047, 4

In [13]:
def main():
    graph = nx.gnm_random_graph(NODES, EDGES)
    memory = ItemMemory()

    initVertices(graph, DIMENSIONS)
    initNodeMem(graph, memory)
    retrain(graph, memory, THRESHOLD, ITER)
    G = initGraph(graph, memory)
    print("G =>", G)
    # testNodeMemoryReconstruction(G, graph, 1)
    # testCheckEdge(G, graph, THRESHOLD)


# main()

In [14]:
graph = nx.gnm_random_graph(NODES, EDGES)
memory = ItemMemory()

initVertices(graph, DIMENSIONS)
initNodeMem(graph, memory)
G = initGraph(graph, memory)
print("G =>", G)

G => [-4.  6.  0. ...  6. 12.  6.]


In [17]:
testNodeMemoryReconstruction(G, graph, 15)

0_00 => 0.2920641517
0_01 => 0.2920641517
0_02 => 0.2920641517
0_03 => 0.2920641517
0_04 => 0.2920641517
0_05 => 0.2920641517
0_06 => 0.2920641517
0_07 => 0.2920641517
0_08 => 0.2920641517
0_09 => 0.2920641517
0_10 => 0.2920641517
0_11 => 0.2920641517
0_12 => 0.2920641517
0_13 => 0.2920641517
0_14 => 0.2920641517


In [16]:
sim(graph.nodes[0]["mem"], graph.nodes[0]["mem"])

1.0

---
